In [1]:
import torch
from transformers import pipeline
import speech_recognition as sr
from gtts import gTTS
import winsound
from pydub import AudioSegment
import pyautogui
import webbrowser
from transformers import BarkModel
from transformers import AutoProcessor
from IPython.display import Audio

In [20]:
import json
import os

In [2]:
# ASR Pipeline
asr_pipe = pipeline("automatic-speech-recognition", model="D:\\digital agent\\whisper-small-cv11-german", device=0)
asr_pipe.model.config.forced_decoder_ids = asr_pipe.tokenizer.get_decoder_prompt_ids(language="de", task="transcribe")
asr_pipe.model.config.max_length = 225 + 1

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
# QA Pipeline
qa_pipe = pipeline(
    "question-answering",
    model="deutsche-telekom/bert-multi-english-german-squad2",
    tokenizer="deutsche-telekom/bert-multi-english-german-squad2"
)

contexts = '''
Deutschland, auch als Bundesrepublik Deutschland bekannt, ist ein faszinierendes Land in Mitteleuropa. Mit einer reichen 
Geschichte, kulturellen Traditionen und einer beeindruckenden Wirtschaft nimmt Deutschland eine herausragende Position in 
der Welt ein. Die deutsche Landschaft ist geprägt von malerischen Flüssen, ausgedehnten Wäldern und charmanten Dörfern. 
Die Städte, wie Berlin, München und Hamburg, zeichnen sich durch ihre architektonische Pracht, pulsierende Kunstszene 
und historischen Denkmäler aus. Die deutsche Küche lockt mit einer Vielzahl von Köstlichkeiten, von deftigen Wurstspezialitäten
bis zu delikaten Backwaren. Mit einer ausgeprägten Kombination aus Tradition und Innovation bleibt Deutschland ein 
faszinierendes Reiseziel für kulturelle Entdeckungen und kulinarische Genüsse.
'''

In [4]:
# TTS Model
tts_model = BarkModel.from_pretrained("D:\\digital agent\\bark-small")
processor = AutoProcessor.from_pretrained("D:\\digital agent\\bark-small")
tts_model = tts_model.to(0)  # Assuming GPU is available



In [17]:
# Function to perform the entire pipeline
def execute_pipeline():
    # ASR Step
    print('Listening.....')
    recognizer = sr.Recognizer()

    with sr.Microphone() as source:
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)#,timeout=10

    try:
        command = recognizer.recognize_google(audio, language="de-DE")
        print("You asked:", command)

        # Save the audio data as a .wav file
        save_path = "asr_audio.wav"
        with open(save_path, "wb") as audio_file:
            audio_file.write(audio.get_wav_data())
    except sr.UnknownValueError:
        print("Could not understand audio. Please try again.")
        return

    # QA Step
    generated_sentences = asr_pipe(save_path)["text"]
    questions = generated_sentences
    answers = qa_pipe(context=contexts, question=questions)
    answer = answers['answer']
    ##answer = 'Ich bin abduallah'

    # TTS Step
    text_prompt = answer
    inputs = processor(text_prompt, voice_preset="de_speaker_1")
    speech_output = tts_model.generate(**inputs.to(0))

    # Display the audio
    sampling_rate = tts_model.generation_config.sample_rate
    display(Audio(speech_output[0].cpu().numpy(), rate=sampling_rate,autoplay=True))



In [19]:
# Execute the pipeline
execute_pipeline()

Listening.....
You asked: was sind einige der bekannte Städte in Deutschland


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [ ]:
'''
Was sind einige der bekannten Städte in Deutschland?
Welche Merkmale prägen die deutsche Landschaft?
Was macht die deutsche Küche besonders?
Warum ist Deutschland ein faszinierendes Reiseziel?
Welche Bedeutung hat Deutschland in der Weltwirtschaft?


Berlin, München und Hamburg sind einige der bekannten Städte in Deutschland.
Die deutsche Landschaft ist geprägt von malerischen Flüssen, ausgedehnten Wäldern und charmanten Dörfern.
Die deutsche Küche lockt mit einer Vielzahl von Köstlichkeiten, von deftigen Wurstspezialitäten bis zu delikaten Backwaren.
Deutschland ist aufgrund seiner reichen Geschichte, kulturellen Traditionen, beeindruckenden Wirtschaft, malerischen Landschaften, pulsierenden Städte und köstlichen Küche ein faszinierendes Reiseziel.
Deutschland nimmt aufgrund seiner beeindruckenden Wirtschaft eine herausragende Position in der Welt ein.
'''